In [1]:
!pip install pymdptoolbox
import numpy as np
import pandas as pd
import mdptoolbox
import datetime
import itertools as itt
from scipy.sparse import csr_matrix

You should consider upgrading via the 'c:\users\deepakraj\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
warehouse_Size = 4
storageCellStates = ["w","r","b","e"] # white, red, blue and empty
actions = ["store_w", "store_r", "store_b", "restore_w", "restore_r", "restore_b"]
states = list(itt.product(storageCellStates, storageCellStates, storageCellStates, storageCellStates, actions))

statesDictionary = {y:j for j,y in enumerate(states)}

positiveRewards = {
    5:2,
    4:4,
    3:4,
    2:6,
    1:6,
    0:8,
    "error": -100
}

negativeRewards = {
    5:-4,
    4:-3,
    3:-3,
    2:-2,
    1:-2,
    0:-1,
    "error": -100
}

rewards = positiveRewards

In [3]:
def get_options(st, act):       #taking state & action ie the index of the cell
                               
    
    viable_state = list()
    valid_state = bool()
    
    ordr = str()
    
    if st[-1][0] == "s":
        ordr = "store"
        valid_state = st[act] == "e"
    else:
        ordr = "restore"
        valid_state = st[act] == st[-1][-1]
    
    if valid_state:
        if ordr == "store":
              viable_state = [j for j,y in enumerate(states) if (y[act] == st[-1][-1]) and ([x for i,x in enumerate(y) if i!=act][:-1] == [x for i,x in enumerate(st) if i!=act][:-1])]
        else:
              viable_state = [j for j,y in enumerate(states) if (y[act] == "e") and ([x for i,x in enumerate(y) if i!=act][:-1] == [x for i,x in enumerate(st) if i!=act][:-1])]
      
    else:
        viable_state = [j for j,y in enumerate(states) if (st[:-1] == y[:-1])]
    
    return viable_state, valid_state

In [4]:
def convert_the_states(s):
    convSt = list()                       #converting indice list to state
    for y in s:
        convSt.append(states[y])    
    return convSt
    print(convSt)

In [5]:
def testing_the_warehouse(order_list, policy):
    st = ["e","e","e","e",""]
  
    cst = 0
  
    for ordr in order_list:
        st[-1] = ordr
        p_cell = policy[statesDictionary[tuple(st)]]
    
        if st[-1][0] == "s":
            st[p_cell] = st[-1][-1]
        else:
            st[p_cell] = "e"
      
        if p_cell == 0:
            cst += 1
        elif p_cell in [1,2]:
            cst += 2
        else:
            cst += 3
    return cst
    print(cst.shape)

In [6]:
def vis_state(st):
    #visualize the structure of a state
    if len(st) == 5:
        print(st[-1])
        print([st[2],st[3]])
        print([st[0],st[1]])
        print("\n")
    else:
        pass

In [7]:
def create_Greedy_Policy():
    pl = list()
  
    for st in states:
        ordr = st[-1][0]
    
        if ordr == "s":
            valid_state = False                               #storing object
            for j in range(4):
                if st[j] == "e":
                                            
                    valid_state = True
                    pl.append(j)
                    break
            if not valid_state:
                                   
                pl.append(3)
        
        else:
                                            #restoring object
            valid_state = False
            for j in range(4):
                if st[j] == st[-1][-1]:
               
                    valid_state = True
                    pl.append(j)
                    break
            if not valid_state:
            
                pl.append(3)
    return pl

In [8]:
#creating transition probability matrices
print(datetime.datetime.now())


transition_prob_matrix = []

probability = 1/6 
probl = 1 - (1/6)*5

for j in range(warehouse_Size):
    transition_probability = np.zeros((1536, 1536))    
    for y in range(len(transition_probability)):
        options, valid_state = get_options(states[y], j)

        if len(options) != 6:
              print("ERROR: wrong options: ", str(len(options)))
          
        for i,k in enumerate(options):
            if i == (len(options)-1):
                  transition_probability[y][k] = probl
            else:
                  transition_probability[y][k] = probability


    transition_prob_matrix.append(csr_matrix(transition_probability))
    print("Done: ", str(j))


print("Probability Transition Matrices created!")


print(transition_prob_matrix)
print(transition_probability)
print(transition_probability.shape)
print(datetime.datetime.now())

2021-07-14 15:23:21.816595
Done:  0
Done:  1
Done:  2
Done:  3
Probability Transition Matrices created!
[<1536x1536 sparse matrix of type '<class 'numpy.float64'>'
	with 9216 stored elements in Compressed Sparse Row format>, <1536x1536 sparse matrix of type '<class 'numpy.float64'>'
	with 9216 stored elements in Compressed Sparse Row format>, <1536x1536 sparse matrix of type '<class 'numpy.float64'>'
	with 9216 stored elements in Compressed Sparse Row format>, <1536x1536 sparse matrix of type '<class 'numpy.float64'>'
	with 9216 stored elements in Compressed Sparse Row format>]
[[0.16666667 0.16666667 0.16666667 ... 0.         0.         0.        ]
 [0.16666667 0.16666667 0.16666667 ... 0.         0.         0.        ]
 [0.16666667 0.16666667 0.16666667 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.16666667 0.16666667 0.16666667]
 [0.         0.         0.         ... 0.16666667 0.16666667 0.16666667]
 [0.         0.         0.         ... 0.1666

In [9]:
#creating reward matrices

reward_matrix = []
for st in states:
    reward = np.zeros(4, dtype=np.float16)
    
    ordr = str()
    if st[-1][0] == "s":
        ordr = "store"
    else:
        ordr = "restore"
    
    if ordr == "store":
        for i,x in enumerate(reward):
            if st[i]=="e":
                reward[i] = rewards[i]
            else:
                reward[i] = rewards["error"]
    else:
        for i,x in enumerate(reward):
            if st[i] == st[-1][-1]:
                reward[i] = rewards[i]
            else:
                reward[i] = rewards["error"]
                
    reward_matrix.append(reward)
    
reward_matrix = np.array(reward_matrix)
print("Rewards matrix created!")
print(reward_matrix)
print(reward_matrix.shape)

Rewards matrix created!
[[-100. -100. -100. -100.]
 [-100. -100. -100. -100.]
 [-100. -100. -100. -100.]
 ...
 [-100. -100. -100. -100.]
 [-100. -100. -100. -100.]
 [-100. -100. -100. -100.]]
(1536, 4)


In [16]:
# Definition of the mdp
mdp_result_Policy = mdptoolbox.mdp.PolicyIteration(transition_prob_matrix, reward_matrix,0.999, max_iter=1000)
mdp_result_Value = mdptoolbox.mdp.ValueIteration(transition_prob_matrix, reward_matrix,0.999, max_iter=1000)
#mdp_result_FH = mdptoolbox.mdp.FiniteHorizon(transition_prob_matrix, 100, reward_matrix, 0.999)
#mdp_result_MDP = classmdptoolbox.mdp.MDP(transition_prob_matrix, reward_matrix,0.999, max_iter=1000, skip_check=False)[source]

In [17]:
print("Runnin Policy")

mdp_result_Policy.run()

print('\nNo of Iterations:\n',mdp_result_Policy.iter)

Runnin Policy

No of Iterations:
 1000


In [18]:
print("Runnin value")

mdp_result_Value.run()
print("MDP trained")

print('\nNo of Iterations:\n',mdp_result_Value.iter)

Runnin value
MDP trained

No of Iterations:
 101


print("FH value")

mdp_result_FH.run()

print("MDP trained")

print('\nNo of Iterations:\n',mdp_result_FH.iter)

In [19]:
print('PolicyIteration:') 
print(mdp_result_Policy.policy) 
print(mdp_result_Policy.V) 
print(mdp_result_Policy.iter)

print('ValueIteration:') 
print(mdp_result_Value.policy) 
print(mdp_result_Value.V) 
print(mdp_result_Value.iter)

PolicyIteration:
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 3, 3, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 3, 3, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 3, 0, 0, 2, 2, 0, 0, 3, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 3, 3, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 3, 3, 0, 3, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 3, 3, 3, 0, 1, 2, 0, 2, 2, 0, 1, 0, 2, 0, 2, 0, 1, 0, 2, 2, 2, 0, 1, 3, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 3, 3, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 3, 3, 3, 0, 2, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 1, 3, 3, 0, 0, 0, 1, 0, 2, 2, 0, 0, 1, 2, 2, 2, 0, 3, 1, 2, 2, 0, 0, 0, 1, 2, 2, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 3, 0, 0, 1, 1, 0, 0, 3, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 1, 1, 0

In [20]:
file1 = open('Exercise 4 - Reinforcement Learning Data - warehouseorder.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    print("Line{}: {}".format(count, line.strip()))

Line1: store	red
Line2: store	blue
Line3: store	white
Line4: restore	blue
Line5: restore	white
Line6: store	white
Line7: store	red
Line8: store	white
Line9: store	red
Line10: restore	red
Line11: restore	red
Line12: store	white
Line13: store	blue
Line14: store	white
Line15: restore	red
Line16: restore	white
Line17: restore	white
Line18: store	red
Line19: store	blue
Line20: restore	red
Line21: store	blue
Line22: store	white
Line23: restore	white
Line24: restore	blue
Line25: restore	white
Line26: restore	white
Line27: restore	blue
Line28: restore	blue
Line29: store	red
Line30: restore	red
Line31: store	red
Line32: store	white
Line33: store	red
Line34: store	red
Line35: store	white
Line36: store	white
Line37: restore	red
Line38: restore	red
Line39: restore	red
Line40: store	red
Line41: restore	white
Line42: restore	red
Line43: restore	white
Line44: store	red
Line45: store	red
Line46: store	white
Line47: store	white
Line48: store	blue
Line49: restore	red
Line50: restore	red
Line51: restore	

print("mdpValue Policy: ",testing_the_warehouse(Lines, mdp_result_Value.policy)) 
print("mdpPolicy Policy: ",testing_the_warehouse(Lines, mdp_result_Policy.policy)) 
print("Greedy Policy: ",testing_the_warehouse(Lines, create_Greedy_Policy()))